In [1]:
# Hadley Wickham - coined the term 'split-apply-combine' for describing group operations

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.159936,0.468937
1,a,two,0.829364,-0.550663
2,b,one,-0.129248,0.600007
3,b,two,-0.846531,0.955671
4,a,one,1.121292,0.353103


In [4]:
# compute mean of the data1 column using the labels from key1
# access data1 and call groupby with the column which is a Series at key1

In [5]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [6]:
# the above grouped variable is now a GroupBy object
# this object has not computed anything yet
# only intermediate data about the group key df['key'] is inferred here
# this object has all the information needed to apply some ops to each of the groups

In [7]:
grouped.mean()

key1
a    0.596907
b   -0.487890
Name: data1, dtype: float64

In [8]:
# here the data has been aggregated as per the group key
# results in a new Series indexed by the unique values in key1 column
# result index has name 'key1' as the DF column df['key1'] has the same name

In [9]:
# pass multiple arrays as a list
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.480678
      two     0.829364
b     one    -0.129248
      two    -0.846531
Name: data1, dtype: float64

In [10]:
# result Series has a hierarchical index with unique pair of keys
means.unstack()

key2,one,two
key1,,
a,0.480678,0.829364
b,-0.129248,-0.846531


In [11]:
# using group keys as arrays of the right length
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [12]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [13]:
df['data1'].groupby([states, years]).mean()

California  2005    0.829364
            2006   -0.129248
Ohio        2005   -0.503234
            2006    1.121292
Name: data1, dtype: float64

In [14]:
# when grouping information is found in the same DF as the data
# pass column names as the group key
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.596907,0.090459
b,-0.487890,0.777839


In [15]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.480678  0.411020
     two   0.829364 -0.550663
b    one  -0.129248  0.600007
     two  -0.846531  0.955671

In [16]:
# size - returns a Series containing group sizes
# returns the count/size of similar elements
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [17]:
# missing values in a group key will be excluded from the result

### ITERATING OVER GROUPS

In [18]:
# GroupBy supports iteration, generating sequence of 2-tuples
# tuples contain the group name and chunk of data

In [19]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.159936  0.468937
1    a  two  0.829364 -0.550663
4    a  one  1.121292  0.353103
b
  key1 key2     data1     data2
2    b  one -0.129248  0.600007
3    b  two -0.846531  0.955671


In [20]:
# in case of multiple keys, 1st element in the tuple will be a tuple of key values
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.159936  0.468937
4    a  one  1.121292  0.353103
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.829364 -0.550663
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.129248  0.600007
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.846531  0.955671


In [21]:
# computing a dict of data pieces as a one-liner
pieces = dict(list(df.groupby('key1')))

In [22]:
pieces['a']

,key1,key2,data1,data2
0,a,one,-0.159936,0.468937
1,a,two,0.829364,-0.550663
4,a,one,1.121292,0.353103


In [23]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.129248,0.600007
3,b,two,-0.846531,0.955671


In [24]:
# groupby groups on axis=0
# grouping the columns by dtype
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [25]:
grouped = df.groupby(df.dtypes, axis=1)

In [26]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.159936  0.468937
1  0.829364 -0.550663
2 -0.129248  0.600007
3 -0.846531  0.955671
4  1.121292  0.353103
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### SELECTING A COLUMN OR SUBSET OF COLUMNS

In [27]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.411020
     two  -0.550663
b    one   0.600007
     two   0.955671

In [28]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [29]:
s_grouped.mean()

key1  key2
a     one     0.411020
      two    -0.550663
b     one     0.600007
      two     0.955671
Name: data2, dtype: float64

### GROUPING WITH DICTS AND SERIES

In [30]:
# grouping information may exist in a form other than array

In [31]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.191963,0.139325,0.244197,-0.076274,0.103755
Steve,-1.700072,-0.984773,1.050445,0.223516,-0.075018
Wes,-0.144251,-1.808186,0.015971,-0.885327,-1.078546
Jim,0.914132,0.863493,-0.774758,-1.281274,0.234940
Travis,-0.702266,-0.220158,1.389560,-0.187102,-1.075944


In [32]:
# adding few NA values
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.191963,0.139325,0.244197,-0.076274,0.103755
Steve,-1.700072,-0.984773,1.050445,0.223516,-0.075018
Wes,-0.144251,NaN,NaN,-0.885327,-1.078546
Jim,0.914132,0.863493,-0.774758,-1.281274,0.234940
Travis,-0.702266,-0.220158,1.389560,-0.187102,-1.075944


In [33]:
# using a group correspondence for columns, sum together the columns by group
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}

In [34]:
# construct an array from the above dict
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.167923,0.435042
Steve,1.273961,-2.759863
Wes,-0.885327,-1.222797
Jim,-2.056032,2.012565
Travis,1.202458,-1.998367


In [35]:
# same funtionalities hold for Series, viewed as a fixed-size mapping
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [36]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### GROUPING WITH FUNCTIONS

In [37]:
# using Python functions is more generic way of defining a group mapping compared with dict or Series
# any function passed as a group key will be called once per index value
# return values will be used as group names

In [38]:
# group by length of the names
people.groupby(len).sum()

,a,b,c,d,e
3,0.961844,1.002817,-0.530561,-2.242875,-0.739851
5,-1.700072,-0.984773,1.050445,0.223516,-0.075018
6,-0.702266,-0.220158,1.389560,-0.187102,-1.075944


In [39]:
# mixing functions with arrays, dicts or Series gets converted to arrays internally
key_list = ['one', 'one', 'one', 'two', 'two']

In [40]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.144251  0.139325  0.244197 -0.885327 -1.078546
  two  0.914132  0.863493 -0.774758 -1.281274  0.234940
5 one -1.700072 -0.984773  1.050445  0.223516 -0.075018
6 two -0.702266 -0.220158  1.389560 -0.187102 -1.075944

### GROUPING BY INDEX LEVELS

In [41]:
# convenience for hierarchical indexed dataset is to aggregate using one of the levels of an axis index

In [42]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

In [43]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -2.630520  0.184320 -0.091586 -0.640857  0.310346
1     -1.237662  0.768355  0.846029 -2.721714 -1.694162
2      0.467134 -1.692333 -0.044069  1.562488 -1.192231
3      2.332614  0.090034 -2.445297 -0.076645 -0.266636

In [44]:
# to group by level, pass the level number or name using the level keyword
hier_df.groupby(level="cty", axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## DATA AGGREGATION

In [45]:
# refers to any data transformation that produces scalar values from arrays
# examples - mean, count, min, sum, etc.
# aggregations can be called on any method that is defined on the grouped object
# example - quantiles computes sample quantiles of a Series or a DF's columns

In [46]:
df

,key1,key2,data1,data2
0,a,one,-0.159936,0.468937
1,a,two,0.829364,-0.550663
2,b,one,-0.129248,0.600007
3,b,two,-0.846531,0.955671
4,a,one,1.121292,0.353103


In [47]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.062906
b   -0.200977
Name: data1, dtype: float64

In [48]:
# to use own aggregation functions, pass any function that aggregates an array to the method
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [49]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.281228,1.019600
b,0.717283,0.355663


In [50]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.596907  0.671501 -0.159936  0.334714  0.829364  0.975328   
b      2.0 -0.487890  0.507195 -0.846531 -0.667210 -0.487890 -0.308569   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.121292   3.0  0.090459  0.558241 -0.550663 -0.098780  0.353103   
b    -0.129248   2.0  0.777839  0.251492  0.600007  0.688923  0.777839   

                          
           75%       max  
key1                      
a     0.411020  0.468937  
b     0.866755  0.955671

### COLUMN-WISE AND MULTIPLE FUNCTION APPLICATION

In [51]:
# load tipping dataset and add a tipping percentage column tip_pct
tips = pd.read_csv('../resources/examples/tips.csv')

In [52]:
# add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [53]:
# easy to aggregate a Series or all columns of DF using aggregate with desired function
# aggregating using a different function on specific column, or using multiple functions at once requires more efforts but doable

In [54]:
# grouping tips by day and smoker
grouped = tips.groupby(['day', 'smoker'])

In [55]:
# for descriptive stats, pass the name of the function as a string
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [56]:
# passing a list of functions will return a DF with column names taken from the functions
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [57]:
# pass a list of (name, function) tuples to be used as the DF column names
# list of 2-tuples as an ordered mapping
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [58]:
# DF has more options, as you can specify a list of functions to apply to all of the columns or different functions per column

In [59]:
functions = ['count', 'mean', 'max']

In [60]:
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [61]:
# the resulting DF has hierarchical columns 
# similar to aggregating each column separately and using concat to glue the results together using column names as key args
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [62]:
# a list of tuples with custom names can be passed

In [63]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [64]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [65]:
# to pass different functions to one or more of the columns
# pass a dict to agg that contains a mapping of column names to any function
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [66]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [67]:
# DF will have hierarchical columns only if multiple functions are applied to at least one column

### RETURNING AGGREGATED DATA WITHOUT ROW INDEXES

In [68]:
# aggregated data comes back with an index which is hierarchical and composed from unique group key combinations
# this can be disabled by passing as_index=False to groupby
# similar can be achieved by reset_index on the result
# using as_index=False method avoids unnecessary computations

In [71]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### APPLY: GENERAL SPLIT-APPLY-COMBINE

In [72]:
# most general purpose GroupBy method is apply
# apply splits the object being manipulated into pieces, then attempts to concatenate the pieces together

In [77]:
# from tipping dataset select the top five tip_pct values by group
# write a function that selects the rows with the largest values in a particular column
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [78]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [79]:
# group by smoker and call apply with this function
# top function is called on each row group from the DF
# results are glued together using pandas.concat
# pieces are labelled with group names
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [81]:
# functions which take in args can be passed after the function
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982